# Model recommendation with lighfm

### Import libraries

In [1]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

### Defining variables

In [2]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [3]:
dtype_df_valid = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"timestampHistory" : 'string'
}

dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [4]:
K_LFM_ITEMS = 2
K_POPULAR_ITEMS = 1
K_ITEMS = 10

### Retrieve data

In [5]:
import pandas as pd
# path config

df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid)
# df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid,nrows=10000)
df_valid.dropna()

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85' '835f...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6' 'b8eb...,[1660548813953 1660572329731 1660594848200]
...,...,...,...,...
112179,aacb28d7d2a4ea745e12ceba1f9ffa0c7b92aae9304ce5...,Non-Logged,['ecb1f348-cd55-47f7-99f9-bb2c84e93f96'],[1660546612592]
112180,91b3a960562e34504022dbe552c0e8947af20fd14ce861...,Non-Logged,['ecb1f348-cd55-47f7-99f9-bb2c84e93f96'],[1660597026440]
112181,9de95a8cd681cd21cc9a7e830859e468b83c051c11f0c4...,Non-Logged,['3d52cd6b-706e-49f0-9215-0340010a9845'],[1660678862844]
112182,001455437b3ca991b144afd8cb785f7727eb0c154faa54...,Non-Logged,['d730c4a6-e8f6-4fde-b73a-afbe148479cd'],[1660584228926]


In [7]:
import pandas as pd

# df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score, nrows=100000)
df_ratings.drop(columns=["Unnamed: 0","score"],inplace=True)
df_ratings

,userId,history,userType
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged
...,...,...,...
99995,6f721d9b4903152bac455dbefbfba7111d1c398619f8af...,d57e382b-d51c-458b-b30e-1236ee1320b9,Logged
99996,6f721d9b4903152bac455dbefbfba7111d1c398619f8af...,47e1ddae-8f6c-41ae-9bdc-6a2acff86bfa,Logged
99997,6f721d9b4903152bac455dbefbfba7111d1c398619f8af...,3b94c985-903b-45db-89a2-24d3d4459509,Logged
99998,6f721d9b4903152bac455dbefbfba7111d1c398619f8af...,e4868445-834a-40b9-a2e2-97b367e58cb0,Logged


In [6]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"])
df_news.drop(columns=["Unnamed: 0"],inplace=True)
df_news.head(3)

,page,age_exp_normalized,ageCategories
0,7371a9b5-5824-4c57-8704-00a74feebe79,0.151439,very-old
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,0.140788,very-old
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,0.122610,very-old


In [7]:
import pickle

loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))
loaded_interactions_shape = pickle.load(open('artifacts/interactions_shape.pkl', 'rb'))

loaded_n_users, loaded_n_items = loaded_interactions_shape

### Make predictions to known and unknowm on same recommendation function with pkls

In [8]:
from utils.custom_treat_data_funcs import explode_df_columns

cols_to_explode = ["history", "timestampHistory"]
cols_and_id = cols_to_explode.copy()
cols_and_id.insert(0,"userId")
cols_and_id.append("userType")
cols_and_id = tuple(cols_and_id)
exploded_df = explode_df_columns(df_valid.loc[:,cols_and_id], cols_to_explode)

In [9]:
df_valid

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85' '835f...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6' 'b8eb...,[1660548813953 1660572329731 1660594848200]
...,...,...,...,...
112179,aacb28d7d2a4ea745e12ceba1f9ffa0c7b92aae9304ce5...,Non-Logged,['ecb1f348-cd55-47f7-99f9-bb2c84e93f96'],[1660546612592]
112180,91b3a960562e34504022dbe552c0e8947af20fd14ce861...,Non-Logged,['ecb1f348-cd55-47f7-99f9-bb2c84e93f96'],[1660597026440]
112181,9de95a8cd681cd21cc9a7e830859e468b83c051c11f0c4...,Non-Logged,['3d52cd6b-706e-49f0-9215-0340010a9845'],[1660678862844]
112182,001455437b3ca991b144afd8cb785f7727eb0c154faa54...,Non-Logged,['d730c4a6-e8f6-4fde-b73a-afbe148479cd'],[1660584228926]


In [10]:
exploded_df

,userId,history,timestampHistory,userType
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,1660533136590,Logged
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,01c59ff6-fb82-4258-918f-2910cb2d4c52,1660672113513,Logged
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,77901133-aee7-4f7b-afc0-652231d76fe9,1660556860253,Logged
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,1660561649242,Logged
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,1660533830245,Logged
...,...,...,...,...
178863,aacb28d7d2a4ea745e12ceba1f9ffa0c7b92aae9304ce5...,ecb1f348-cd55-47f7-99f9-bb2c84e93f96,1660546612592,Non-Logged
178864,91b3a960562e34504022dbe552c0e8947af20fd14ce861...,ecb1f348-cd55-47f7-99f9-bb2c84e93f96,1660597026440,Non-Logged
178865,9de95a8cd681cd21cc9a7e830859e468b83c051c11f0c4...,3d52cd6b-706e-49f0-9215-0340010a9845,1660678862844,Non-Logged
178866,001455437b3ca991b144afd8cb785f7727eb0c154faa54...,d730c4a6-e8f6-4fde-b73a-afbe148479cd,1660584228926,Non-Logged


In [11]:
exploded_df["check_hifens"] = exploded_df["history"].str.split('-').apply(lambda x : len(x) == 5).astype("bool")
exploded_df["check_size"] = exploded_df["history"].str.replace(r'-', '', regex=True).apply(lambda x : len(x) == 32).astype("bool")
exploded_df["check_chars"] = exploded_df["history"].str.replace(r'-', '', regex=True).str.replace(r'[a-f0-9]', '', regex=True).apply(lambda x : x == '').astype("bool")
exploded_df["check_history"] = (exploded_df["check_hifens"] & exploded_df["check_size"] & exploded_df["check_chars"]).astype("bool")
exploded_df["check_history"].value_counts()

check_history
True     172442
False      6426
Name: count, dtype: int64

In [12]:
exploded_df_valid = exploded_df[~exploded_df["check_history"]==False][["userId","userType","history"]]
exploded_df_valid

,userId,userType,history
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85
...,...,...,...
178863,aacb28d7d2a4ea745e12ceba1f9ffa0c7b92aae9304ce5...,Non-Logged,ecb1f348-cd55-47f7-99f9-bb2c84e93f96
178864,91b3a960562e34504022dbe552c0e8947af20fd14ce861...,Non-Logged,ecb1f348-cd55-47f7-99f9-bb2c84e93f96
178865,9de95a8cd681cd21cc9a7e830859e468b83c051c11f0c4...,Non-Logged,3d52cd6b-706e-49f0-9215-0340010a9845
178866,001455437b3ca991b144afd8cb785f7727eb0c154faa54...,Non-Logged,d730c4a6-e8f6-4fde-b73a-afbe148479cd


In [13]:
count_histories = exploded_df_valid.groupby(["history"]).size().sort_values(ascending=False).astype(dtype="UInt16")
count_histories

history
9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6    7351
4c3d47a1-6f4b-424f-8944-6c227e686c5c    5036
eb23272d-8e6c-479d-b972-eabeb5f6f3dd    4226
d730c4a6-e8f6-4fde-b73a-afbe148479cd    2963
be89a7da-d9fa-49d4-9fdc-388c27a15bc8    2921
                                        ... 
ffc68ca1-6d6b-4dc5-b7f4-5cb446d4c4b0       1
0042a36e-a7eb-46a5-83b4-bff52fe717da       1
ffdc3cc8-050a-4823-a325-b68ad358b5ec       1
0020438f-4ef6-4f93-a43a-7f59254e740e       1
0026f5f4-dd10-4b9c-ae41-7c1734710795       1
Length: 36187, dtype: UInt16

In [14]:
top_k_popular_histories = count_histories.head(K_POPULAR_ITEMS)
top_k_popular_histories = list(set(top_k_popular_histories.keys()))

In [15]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [16]:
def sample_recommendation_by_title(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    
    top_k_indices = np.argsort(-scores)[:K_LFM_ITEMS]  # Sort scores in descending order and take the top K_ITEMS
    top_k_items_lfm = [item_id_map_reverse[i] for i in top_k_indices]
    top_k_items = list(set(top_k_items_lfm + top_k_popular_histories))

    print("Top 5 recommended items:")

    for x in top_k_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["title"].values[0])


In [17]:
def get_recommended_history_list(user_hash,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    """
    This function verifies if the users is known or new, and makes recommendations depending on this verification.
    The top 5 recommendations from the list are returned.
    """
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    
    top_k_indices = np.argsort(-scores)[:K_LFM_ITEMS]  # Sort scores in descending order and take the top K_ITEMS
    top_k_items_lfm = [item_id_map_reverse[i] for i in top_k_indices]
    top_k_items = list(set(top_k_items_lfm + top_k_popular_histories))

    return top_k_items

In [18]:
# # Testing for just one user
# user = 38
# user_feature_list = [df_valid["userType"].iloc[user]]
# user_hash = df_valid["userId"].iloc[user]
# validation_history_hashes = df_valid["history"].iloc[user]

# recommeded_histories = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
# print(recommeded_histories)
# print(validation_history_hashes)
# print(validation_history_hashes in recommeded_histories)

In [19]:
def count_valid_recommendations(validation_history_hashes, recommeded_histories):
    """
    This function receives 
    * The `validation_history_hashes` (which is a list of histories contained in the "validacao.csv")
    * The `recommeded_histories` (which are the recommended histories/items by the model)
    Then, it verifies how many recommended items match with the validation histories.
    """
    count_valids = 0
    for valid_hist in validation_history_hashes:
        if valid_hist in recommeded_histories:
            count_valids = count_valids+1
    return count_valids

In [20]:
# for index, row in df_valid.iterrows():
#     """
#     For each user, get the top 5 recommendations from the model, and also count how many of them are contained within the validation set.
#     Stores the results on the columns `recommended_hists` `matched_recommendations`.
#     """
#     user_feature_list = [row["userType"]]
#     user_hash = row["userId"]
#     validation_history_hashes = row["history"]

#     recommended_hist = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
#     num_valid_recommendations = count_valid_recommendations(validation_history_hashes, recommended_hist)

#     row["recommended_hists"] = recommended_hist
#     row["matched_recommendations"] = num_valid_recommendations

In [21]:
exploded_df_valid = exploded_df_valid.head(1000)

exploded_df_valid.loc[:,"recommended_hists"] = exploded_df_valid.apply(lambda x :
    get_recommended_history_list(
        x.userId,
        [x.userType],
        loaded_item_id_map_reverse,
        loaded_user_feature_map,
        loaded_user_id_map,
        loaded_model),
        axis=1)

/tmp/ipykernel_444826/1618428163.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exploded_df_valid.loc[:,"recommended_hists"] = exploded_df_valid.apply(lambda x :


In [22]:
exploded_df_valid["matched_recommendations"] = exploded_df_valid.apply(lambda x :
    count_valid_recommendations(x.history, x.recommended_hists),
    axis=1
    )

/tmp/ipykernel_444826/2697419088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exploded_df_valid["matched_recommendations"] = exploded_df_valid.apply(lambda x :


In [23]:
exploded_df_valid.describe()

,matched_recommendations
count,1000.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [24]:
exploded_df_valid

,userId,userType,history,recommended_hists,matched_recommendations
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,"[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, ef2eb37...",0
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52,"[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, ef2eb37...",0
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9,"[b5f56df6-36a4-4956-a926-eeeb6950fc3c, b7cf538...",0
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,"[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, f10ca1a...",0
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,"[ddc7b7d1-5c15-4ebe-a267-a91c489a42f4, 9c764c3...",0
...,...,...,...,...,...
1029,a98d150b8cf967b0fcb8d117a4e5531ab6f051595053c4...,Logged,be81f38c-fc94-4d30-90cc-4136b01dc83d,"[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, 1298fd3...",0
1030,0c564af27eaac70be62f377fc681e2e3236b58a9d57ff8...,Logged,9d3770aa-e8bb-44a6-b2bb-ac511bf1033a,"[b5f56df6-36a4-4956-a926-eeeb6950fc3c, 9c764c3...",0
1031,0c564af27eaac70be62f377fc681e2e3236b58a9d57ff8...,Logged,72014099-6ee5-4c3a-9f96-7ed0b7461edb,"[b5f56df6-36a4-4956-a926-eeeb6950fc3c, 9c764c3...",0
1032,73d4fd880933325b82bed1890b7f9322a9222e40e5440c...,Logged,72014099-6ee5-4c3a-9f96-7ed0b7461edb,"[4eca7691-e441-4c06-89db-a2e18fd8198e, 9c764c3...",0


In [25]:
user = 1
user_feature_list = [exploded_df_valid["userType"].iloc[user]]
user_hash = exploded_df_valid["userId"].iloc[user]
validation_history_hashes = exploded_df_valid["history"].iloc[user]

print(user_hash)
print(user_feature_list)

recommeded_histories = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
print(recommeded_histories)
print(validation_history_hashes)
print(validation_history_hashes in recommeded_histories)

e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9
['Logged']
['b7cf538b-7fbc-42ca-ba46-f003db7efeaa', 'ef2eb378-4f0c-42e5-a196-704d3a795a25', '9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6']
01c59ff6-fb82-4258-918f-2910cb2d4c52
False


In [26]:
exploded_df_valid[exploded_df_valid["matched_recommendations"]>0]

,userId,userType,history,recommended_hists,matched_recommendations


In [27]:
exploded_df_valid

,userId,userType,history,recommended_hists,matched_recommendations
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,"[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, ef2eb37...",0
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52,"[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, ef2eb37...",0
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9,"[b5f56df6-36a4-4956-a926-eeeb6950fc3c, b7cf538...",0
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,"[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, f10ca1a...",0
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,"[ddc7b7d1-5c15-4ebe-a267-a91c489a42f4, 9c764c3...",0
...,...,...,...,...,...
1029,a98d150b8cf967b0fcb8d117a4e5531ab6f051595053c4...,Logged,be81f38c-fc94-4d30-90cc-4136b01dc83d,"[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, 1298fd3...",0
1030,0c564af27eaac70be62f377fc681e2e3236b58a9d57ff8...,Logged,9d3770aa-e8bb-44a6-b2bb-ac511bf1033a,"[b5f56df6-36a4-4956-a926-eeeb6950fc3c, 9c764c3...",0
1031,0c564af27eaac70be62f377fc681e2e3236b58a9d57ff8...,Logged,72014099-6ee5-4c3a-9f96-7ed0b7461edb,"[b5f56df6-36a4-4956-a926-eeeb6950fc3c, 9c764c3...",0
1032,73d4fd880933325b82bed1890b7f9322a9222e40e5440c...,Logged,72014099-6ee5-4c3a-9f96-7ed0b7461edb,"[4eca7691-e441-4c06-89db-a2e18fd8198e, 9c764c3...",0


In [28]:
exploded_df_valid["recommended_hists"].value_counts()

recommended_hists
[4eca7691-e441-4c06-89db-a2e18fd8198e, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6, b3183ccd-e1b3-4e86-b582-ec719eba6a1a]    99
[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, 4eca7691-e441-4c06-89db-a2e18fd8198e, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6]    67
[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, f10ca1ae-924d-472f-aff8-8d210867bb05, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6]    48
[b58e7ad3-dae4-4d49-810d-85569568833c, b7cf538b-7fbc-42ca-ba46-f003db7efeaa, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6]    37
[b7cf538b-7fbc-42ca-ba46-f003db7efeaa, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6, b3183ccd-e1b3-4e86-b582-ec719eba6a1a]    36
                                                                                                                      ..
[df9aee0d-9bcb-4108-b136-67e97071db38, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6, 42a8b32e-d6e4-4296-87e0-2e8f392a8915]     1
[9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6, cfc011b6-7b1a-4c16-a368-17c17306c97e, 64811d69-ca77-4da9-8dff-ff374aea9856]     1
[ef2eb378-4f0c